# Web Crawler 

***

## 用途
 - 爬蟲是透過程式**自動抓取**網站資料的過程，在資訊量爆炸的現在，資料蒐集是一個相當重要的工作，如果依舊透過人工的方式來蒐集網站資料，除了效率差外還會浪費掉很多的時間。
 - 因此就可以透過爬蟲協助，只要我們先定好規則，讓爬蟲來自動化蒐集與擷取資料，整理出我們所需要的內容。

## 主要流程
1. 傳送GET請求給目標網站
2. 抓取所需資料
    - 將HTML文件 **解析** 並 **擷取** 需要的資料
3. 儲存資料
    - 將擷取整理過的資料儲存在CSV、Excel或是資料庫。

## 使用工具
1. ***BeautifulSoup***: 用於解析網頁的套件
2. ***Selenium***: 用於抓取透過JavaScript產生的動態網頁的內容


***

## 安裝環境

## ```pip install requests```
### 對網路發動請求的套件，可實作對網頁做get、post等HTTP協定的行為

## ```pip install beautifulsoup4```
### 借助網頁的結構特性來解析網頁的工具，只需要簡單的幾條指令就可以提取HTML標籤裡的元素

***

## 實作1: 爬取PTT八卦版的文章資訊

### 以PTT八卦版為例 : https://www.ptt.cc/bbs/Gossiping/index.html

### Step1. import 套件

In [1]:
import requests
from bs4 import BeautifulSoup

# 把Warning訊息去掉
requests.packages.urllib3.disable_warnings()

***

### Step2. 將網頁Get下來

In [2]:
# 設定proxy 在公司內需使用
http_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
https_proxy = "http://auhqproxy.corpnet.auo.com:8080/"
ftp_proxy = "http://auhqproxy.corpnet.auo.com:8080/"

proxyDict = { 
              "http"  : http_proxy, 
              "https" : https_proxy, 
              "ftp"   : ftp_proxy
            }
            
#r = requests.get(url, proxies=proxyDict, verify=False)

In [3]:
# 要爬的網站
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

***

### Step3. 處理PTT網站詢問是否已滿18歲的問題
1. 進入網站後會看到讓使用者點選「是否已滿18歲」按鈕

2. 可開啟瀏覽器開發者模式(F12)，並點選至network(網路)，觀察點選「已滿18歲」後，會送給伺服器之封包內容。

3. 發現封包以POST發送，並且封包內容有cookie和參數

4. 我們的程式碼可以透過POST的方式，送出cookie和特定的參數

5. 可以藉由requests的一個參數來儲存自身的cookie
   Session會將你送出的requests所收到的cookies全部儲存起來並且在發送下一次請求時送出對應的參數。

In [4]:
# 處理是否已滿18歲的問題
payload = {
    "from":"/bbs/Gossiping/index.html",
    "yes":"yes"
}

rs = requests.session()

# 送出資訊已滿18歲的資訊
# verify=False 是為了避免SSL認證
res = rs.post("https://www.ptt.cc/ask/over18", proxies=proxyDict, verify=False, data=payload)

# 將此頁面的HTML GET下來
res = rs.get(url, proxies=proxyDict, verify=False)

# 印出HTML
print(res.text)

<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print">




	<script async src='/cdn-cgi/challenge-platform/h/g/scripts/invisible.js?ts=1650610800'></script></head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossi

***

### Step4. 以HTML標籤及屬性搜尋節點

### soup物件包含了整個網頁的HTML程式碼，接下來利用BeautifulSoup套件進行節點的搜尋。

In [5]:
# 將網頁資料以html.parser
soup = BeautifulSoup(res.text, "html.parser")

### 1. find()
    - 只搜尋第一個符合條件的HTML節點，傳入要搜尋的標籤名稱

In [6]:
result = soup.find("div")
print(result)

<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>


### 2. find_all()
    - 搜尋所有符合條件的HTML節點，傳入要搜尋的標籤名稱
    - 可以利用關鍵字參數(Keyword Argument)指定其屬性值。
    - 由於執行結果可能會搜出許多的HTML內容，所以最後也可以利用limit關鍵字參數(Keyword Argument)限制搜尋的節點數量

In [7]:
result = soup.find_all("div", id="action-bar-container", limit=3)
print(result)

[<div id="action-bar-container">
<div class="action-bar">
<div class="btn-group btn-group-dir">
<a class="btn selected" href="/bbs/Gossiping/index.html">看板</a>
<a class="btn" href="/man/Gossiping/index.html">精華區</a>
</div>
<div class="btn-group btn-group-paging">
<a class="btn wide" href="/bbs/Gossiping/index1.html">最舊</a>
<a class="btn wide" href="/bbs/Gossiping/index39346.html">‹ 上頁</a>
<a class="btn wide disabled">下頁 ›</a>
<a class="btn wide" href="/bbs/Gossiping/index.html">最新</a>
</div>
</div>
</div>]


### 2.1 同時搜尋多個HTML標籤

In [8]:
result = soup.find_all(["div", "a"], limit=5)
print(result)

[<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>, <div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>, <a href="/bbs/" id="logo">批踢踢實業坊</a>, <a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>, <a class="right small" href="/about.html">關於我們</a>]


### 3. 搜尋子節點
### select_one()
    - 當某一節點下只有單個子節點時，可以利用select_one()
### select()
    - 當某一節點下有多個子節點時，可以利用select()

In [9]:
result = soup.find("div")

In [10]:
print(result)

<div id="topbar-container">
<div class="bbs-content" id="topbar">
<a href="/bbs/" id="logo">批踢踢實業坊</a>
<span>›</span>
<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
<a class="right small" href="/about.html">關於我們</a>
<a class="right small" href="/contact.html">聯絡資訊</a>
</div>
</div>


In [11]:
print(result.select_one("a"))

<a href="/bbs/" id="logo">批踢踢實業坊</a>


In [12]:
print(result.select("a"))

[<a href="/bbs/" id="logo">批踢踢實業坊</a>, <a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>, <a class="right small" href="/about.html">關於我們</a>, <a class="right small" href="/contact.html">聯絡資訊</a>]


***

### Step 5. 以CSS屬性搜尋節點
    - 依據HTML的css屬性來進行節點的搜尋，需使用class_ 進行css屬性值的指定

### 1. find()
    - 搜尋第一個符合指定的HTML標籤及css屬性值的節點

In [13]:
result = soup.find("div", class_="r-ent")
print(result)

<div class="r-ent">
<div class="nrec"></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650612668.A.665.html">[新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續</a>
</div>
<div class="meta">
<div class="author">Skyblueway</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E4%B8%8D%E5%8F%AA3%E5%8D%83%E8%90%AC%E5%8A%91%E5%BF%AB%E7%AF%A9%E4%B8%8B%E5%91%A8%E5%88%B0%E8%B2%A8%E3%80%80%E9%99%B3%E6%99%82%E4%B8%AD%EF%BC%9A%E9%99%B8%E7%BA%8C">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3ASkyblueway">搜尋看板內 Skyblueway 的文章</a></div>
</div>
</div>
<div class="date"> 4/22</div>
<div class="mark"></div>
</div>
</div>


### 2. find_all()
    - 搜尋網頁中符合指定的HTML標籤及css屬性值的所有節點

In [14]:
result = soup.find_all("div", class_="r-ent", limit=3)
print(result)

[<div class="r-ent">
<div class="nrec"></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650612668.A.665.html">[新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續</a>
</div>
<div class="meta">
<div class="author">Skyblueway</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E4%B8%8D%E5%8F%AA3%E5%8D%83%E8%90%AC%E5%8A%91%E5%BF%AB%E7%AF%A9%E4%B8%8B%E5%91%A8%E5%88%B0%E8%B2%A8%E3%80%80%E9%99%B3%E6%99%82%E4%B8%AD%EF%BC%9A%E9%99%B8%E7%BA%8C">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3ASkyblueway">搜尋看板內 Skyblueway 的文章</a></div>
</div>
</div>
<div class="date"> 4/22</div>
<div class="mark"></div>
</div>
</div>, <div class="r-ent">
<div class="nrec"><span class="hl f2">1</span></div>
<div class="title">
			
				(本文已被刪除) [golang]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/22</div>
<div class="mar

### 3. select()
    - 單純只想要透過css屬性值來進行HTML節點的搜尋

In [15]:
result = soup.select(".r-ent", limit=3)
print(result)

[<div class="r-ent">
<div class="nrec"></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650612668.A.665.html">[新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續</a>
</div>
<div class="meta">
<div class="author">Skyblueway</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E4%B8%8D%E5%8F%AA3%E5%8D%83%E8%90%AC%E5%8A%91%E5%BF%AB%E7%AF%A9%E4%B8%8B%E5%91%A8%E5%88%B0%E8%B2%A8%E3%80%80%E9%99%B3%E6%99%82%E4%B8%AD%EF%BC%9A%E9%99%B8%E7%BA%8C">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3ASkyblueway">搜尋看板內 Skyblueway 的文章</a></div>
</div>
</div>
<div class="date"> 4/22</div>
<div class="mark"></div>
</div>
</div>, <div class="r-ent">
<div class="nrec"><span class="hl f2">1</span></div>
<div class="title">
			
				(本文已被刪除) [golang]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/22</div>
<div class="mar

***

### Step 6. 搜尋父節點
    - 以上皆為向下的搜尋節點方式，如果想要從某一個節點向上搜尋，可以透過find_parent()跟find_parents()

In [16]:
result = soup.find("div", class_="title")
parents = result.find_parents("div")
print(parents)

[<div class="r-ent">
<div class="nrec"></div>
<div class="title">
<a href="/bbs/Gossiping/M.1650612668.A.665.html">[新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續</a>
</div>
<div class="meta">
<div class="author">Skyblueway</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E4%B8%8D%E5%8F%AA3%E5%8D%83%E8%90%AC%E5%8A%91%E5%BF%AB%E7%AF%A9%E4%B8%8B%E5%91%A8%E5%88%B0%E8%B2%A8%E3%80%80%E9%99%B3%E6%99%82%E4%B8%AD%EF%BC%9A%E9%99%B8%E7%BA%8C">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3ASkyblueway">搜尋看板內 Skyblueway 的文章</a></div>
</div>
</div>
<div class="date"> 4/22</div>
<div class="mark"></div>
</div>
</div>, <div class="r-list-container action-bar-margin bbs-screen">
<div class="search-bar">
<form action="search" id="search-bar" type="get">
<input class="query" name="q" placeholder="搜尋文章⋯" type="text" value=""/>
</form>
</div>
<div class="r-ent">
<div class="nrec

***

### Step 7. 搜尋前、後節點
    - 在同一層級的節點，想要搜尋前一個節點，可以透過find_previous_siblings()
    - 在同一層級的節點，想要搜尋後一個節點，可以透過find_next_siblings()

In [17]:
#print(res.text)

In [18]:
result = soup.find("div", class_="title")
previous_node = result.find_previous_siblings("div")
print(previous_node)

[<div class="nrec"></div>]


In [19]:
result = soup.find("div", class_="title")
previous_node = result.find_next_siblings("div")
print(previous_node)

[<div class="meta">
<div class="author">Skyblueway</div>
<div class="article-menu">
<div class="trigger">⋯</div>
<div class="dropdown">
<div class="item"><a href="/bbs/Gossiping/search?q=thread%3A%5B%E6%96%B0%E8%81%9E%5D+%E4%B8%8D%E5%8F%AA3%E5%8D%83%E8%90%AC%E5%8A%91%E5%BF%AB%E7%AF%A9%E4%B8%8B%E5%91%A8%E5%88%B0%E8%B2%A8%E3%80%80%E9%99%B3%E6%99%82%E4%B8%AD%EF%BC%9A%E9%99%B8%E7%BA%8C">搜尋同標題文章</a></div>
<div class="item"><a href="/bbs/Gossiping/search?q=author%3ASkyblueway">搜尋看板內 Skyblueway 的文章</a></div>
</div>
</div>
<div class="date"> 4/22</div>
<div class="mark"></div>
</div>]


***

### Step 8. 取得屬性值
    - 如果想要取得某一個節點中的屬性值，則可以利用BeautifulSoup套件的get()

In [20]:
result = soup.find_all("div", class_="r-ent")
for title in result:
    print(title.select_one("a"))

<a href="/bbs/Gossiping/M.1650612668.A.665.html">[新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續</a>
None
<a href="/bbs/Gossiping/M.1650612703.A.0A3.html">Re: [新聞] 獨家｜欠20萬償300萬還不夠！特戰士官不</a>
<a href="/bbs/Gossiping/M.1650612732.A.426.html">[新聞] 「以篩代隔」隔離保單賠不賠？ 業者說</a>
<a href="/bbs/Gossiping/M.1650612740.A.15A.html">[問卦] 養狗還是養小孩?</a>
<a href="/bbs/Gossiping/M.1650612824.A.CDE.html">Re: [問卦] 台灣的大學生到底在怕甚麼?????</a>
<a href="/bbs/Gossiping/M.1650612863.A.F2F.html">[問卦] 用無人機外送可行嗎？</a>
<a href="/bbs/Gossiping/M.1650612898.A.B1F.html">[問卦] 是不是要等年輕人死光了再來打疫苗?</a>
<a href="/bbs/Gossiping/M.1650612930.A.705.html">Re: [爆卦] 本土+3766 類官方粗暴版本（轉）</a>
<a href="/bbs/Gossiping/M.1650612986.A.0A8.html">[新聞] 雙北逾3萬人居隔！打3劑｢以篩代隔｣擬擴</a>
<a href="/bbs/Gossiping/M.1650612998.A.5F5.html">Re: [問卦] 台鐵員工5/1憑啥以勞工名義合理罷工</a>
<a href="/bbs/Gossiping/M.1650613014.A.663.html">[問卦] 何不用氣球將病患送醫跟送物資就好？</a>
<a href="/bbs/Gossiping/M.1650613015.A.431.html">[問卦] 老闆反應永遠慢半拍的八卦？</a>
<a href="/bbs/Gossiping/M.1650613023.A.974.html">[問卦] 為什麼科技版那麼多神祕數字？</a>
<a

In [21]:
result = soup.find_all("div", class_="r-ent")
for title in result:
    if title.select_one("a") != None:
        print(title.select_one("a").get("href"), " : ", title.select_one("a").text)
        #print(title.select_one("a").get("href"), " : ", title.select_one("a").getText())

/bbs/Gossiping/M.1650612668.A.665.html  :  [新聞] 不只3千萬劑快篩下周到貨　陳時中：陸續
/bbs/Gossiping/M.1650612703.A.0A3.html  :  Re: [新聞] 獨家｜欠20萬償300萬還不夠！特戰士官不
/bbs/Gossiping/M.1650612732.A.426.html  :  [新聞] 「以篩代隔」隔離保單賠不賠？ 業者說
/bbs/Gossiping/M.1650612740.A.15A.html  :  [問卦] 養狗還是養小孩?
/bbs/Gossiping/M.1650612824.A.CDE.html  :  Re: [問卦] 台灣的大學生到底在怕甚麼?????
/bbs/Gossiping/M.1650612863.A.F2F.html  :  [問卦] 用無人機外送可行嗎？
/bbs/Gossiping/M.1650612898.A.B1F.html  :  [問卦] 是不是要等年輕人死光了再來打疫苗?
/bbs/Gossiping/M.1650612930.A.705.html  :  Re: [爆卦] 本土+3766 類官方粗暴版本（轉）
/bbs/Gossiping/M.1650612986.A.0A8.html  :  [新聞] 雙北逾3萬人居隔！打3劑｢以篩代隔｣擬擴
/bbs/Gossiping/M.1650612998.A.5F5.html  :  Re: [問卦] 台鐵員工5/1憑啥以勞工名義合理罷工
/bbs/Gossiping/M.1650613014.A.663.html  :  [問卦] 何不用氣球將病患送醫跟送物資就好？
/bbs/Gossiping/M.1650613015.A.431.html  :  [問卦] 老闆反應永遠慢半拍的八卦？
/bbs/Gossiping/M.1650613023.A.974.html  :  [問卦] 為什麼科技版那麼多神祕數字？
/bbs/Gossiping/M.1650613023.A.DA6.html  :  [新聞] 老舊公寓居家照顧怎麼下樓取藥？陳時中：
/bbs/Gossiping/M.1650613042.A.5DF.html  :  Re: [新聞] 北市4/25起國高中遠距教學 新北